In [2]:
from dataset import Kitti, get_dataloader
import pandas as pd
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import os
import subprocess

from setuptools import find_packages, setup
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

def make_cuda_ext(name, module, sources):
    cuda_ext = CUDAExtension(
        name='%s.%s' % (module, name),
        sources=[os.path.join(*module.split('.'), src) for src in sources]
    )
    return cuda_ext

In [4]:
make_cuda_ext(
                name='roiaware_pool3d_cuda',
                module='pcdet.ops.roiaware_pool3d',
                sources=[
                    'src/roiaware_pool3d.cpp',
                    'src/roiaware_pool3d_kernel.cu',
                ]
            )

<setuptools.extension.Extension('pcdet.ops.roiaware_pool3d.roiaware_pool3d_cuda') at 0x7fb2780ae8e0>

In [54]:
kitti_data = Kitti(data_root='data/kitti', 
                    split='train')
example = kitti_data.__getitem__(9)
example['pts'].shape
box = example['gt_bboxes_3d'][0]

In [ ]:
def restore_corners():
    pass

In [20]:
example['gt_bboxes_3d']

array([[ 41.019917  , -15.135154  ,  -1.2213999 ,   1.4423676 ,
          4.3271027 ,   1.286436  ,  -1.4590387 ],
       [ 19.59957   , -19.58695   ,  -1.5335454 ,   2.5338888 ,
         16.363075  ,   3.917782  ,   1.8608462 ],
       [ 31.678396  ,  -9.892149  ,  -1.336067  ,   1.5203333 ,
          4.054222  ,   1.3838931 ,  -1.4863529 ],
       [  4.4121037 ,  -6.1468782 ,  -1.4431155 ,   1.5495706 ,
          3.35253   ,   1.4131304 ,  -1.3364711 ],
       [ 38.27281   , -25.586308  ,  -1.0381042 ,   1.3156731 ,
          3.1478696 ,   1.4033847 ,   1.9633679 ],
       [ 18.263111  ,  -5.094483  ,  -1.4047003 ,   1.6177906 ,
          3.9762564 ,   1.3644017 ,  -1.1407828 ],
       [ 27.382992  , -10.002426  ,  -1.1761512 ,   1.5495706 ,
          3.6936305 ,   1.461859  ,  -1.1037278 ],
       [ 46.25018   ,   1.2323573 ,  -0.79460055,   1.6567736 ,
          3.810579  ,   1.5982991 ,  -1.1020136 ],
       [ 32.95406   , -12.903475  ,  -1.3473883 ,   1.5008419 ,
          3.7618

In [24]:
def bbox3d2corners(bboxes):
    '''
    bboxes: shape=(n, 7)
    return: shape=(n, 8, 3)
           ^ z   x            6 ------ 5
           |   /             / |     / |
           |  /             2 -|---- 1 |   
    y      | /              |  |     | | 
    <------|o               | 7 -----| 4
                            |/   o   |/    
                            3 ------ 0 
    x: front, y: left, z: top
    '''
    centers, dims, angles = bboxes[:, :3], bboxes[:, 3:6], bboxes[:, 6]

    # 1.generate bbox corner coordinates, clockwise from minimal point
    bboxes_corners = np.array([[-0.5, -0.5, 0], [-0.5, -0.5, 1.0], [-0.5, 0.5, 1.0], [-0.5, 0.5, 0.0],
                               [0.5, -0.5, 0], [0.5, -0.5, 1.0], [0.5, 0.5, 1.0], [0.5, 0.5, 0.0]], 
                               dtype=np.float32)
    bboxes_corners = bboxes_corners[None, :, :] * dims[:, None, :] # (1, 8, 3) * (n, 1, 3) -> (n, 8, 3)

    # 2. rotate around z axis
    rot_sin, rot_cos = np.sin(angles), np.cos(angles)
    # in fact, -angle
    rot_mat = np.array([[rot_cos, rot_sin, np.zeros_like(rot_cos)],
                        [-rot_sin, rot_cos, np.zeros_like(rot_cos)],
                        [np.zeros_like(rot_cos), np.zeros_like(rot_cos), np.ones_like(rot_cos)]], 
                        dtype=np.float32) # (3, 3, n)
    rot_mat = np.transpose(rot_mat, (2, 1, 0)) # (n, 3, 3)
    bboxes_corners = bboxes_corners @ rot_mat # (n, 8, 3)

    # 3. translate to centers
    bboxes_corners += centers[:, None, :]
    return bboxes_corners

In [25]:
bbox3d2corners(example['gt_bboxes_3d']).shape

(31, 8, 3)

In [ ]:
def remove_pts_in_bboxes(points, bboxes, rm=True):
    '''
    points: shape=(N, 3)
    bboxes: shape=(n, 7)
    return: shape=(N, n), bool
    '''
    # 1. get 6 groups of rectangle vertexs
    bboxes_corners = bbox3d2corners(bboxes) # (n, 8, 3)
    bbox_group_rectangle_vertexs = group_rectangle_vertexs(bboxes_corners) # (n, 6, 4, 3)

    # 2. calculate plane equation: ax + by + cd + d = 0
    group_plane_equation_params = group_plane_equation(bbox_group_rectangle_vertexs)

    # 3. Judge each point inside or outside the bboxes
    # if point (x0, y0, z0) lies on the direction of normal vector(a, b, c), then ax0 + by0 + cz0 + d > 0.
    masks = points_in_bboxes(points, group_plane_equation_params) # (N, n)

    if not rm:
        return masks
        
    # 4. remove point insider the bboxes
    masks = np.any(masks, axis=-1)

    return points[~masks]

In [8]:
from pcdet.utils.box_utils import boxes_to_corners_3d, remove_points_in_boxes3d

In [10]:
len(remove_points_in_boxes3d(example['pts'], example['gt_bboxes_3d']))

9392

In [30]:

corner_pts = boxes_to_corners_3d(example['gt_bboxes_3d'])
idx_octodron = np.random.randint(8, size=corner_pts.shape[0])
corner_picked = corner_pts[np.arange(corner_pts.shape[0]), idx_octodron]
new_centers = (example['gt_bboxes_3d'][:, :3] + corner_picked) / 2
new_dims = example['gt_bboxes_3d'][:, 3:-1] / 2

In [58]:
filter_bboxes = np.hstack((new_centers, new_dims, example['gt_bboxes_3d'][:, -1:]))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 29 and the array at index 2 has size 26

In [63]:
def octodron_dropout(data_dict):
    pts, gt_bboxes_3d = data_dict['pts'], data_dict['gt_bboxes_3d']

    gt_bbox3d_corner_pts = boxes_to_corners_3d(gt_bboxes_3d)

    # randomly pick for each gt bbox one of the 8 octodrons to dropout
    idx_octodron = np.random.randint(8, size=gt_bbox3d_corner_pts.shape[0])
    corners_picked = gt_bbox3d_corner_pts[np.arange(gt_bbox3d_corner_pts.shape[0]), idx_octodron]

    # construct the new centroids and new h, l, w dims of the filtering 3d bboxes
    # the yaw around z axis remains unchanged
    new_centers = (gt_bboxes_3d[:, :3] + corners_picked) / 2
    new_dims = gt_bboxes_3d[:, 3:-1] / 2
    
    filter_bboxes = np.hstack((new_centers, new_dims, gt_bboxes_3d[:, -1:]))
    data_dict['pts'] = remove_points_in_boxes3d(pts, filter_bboxes)

    return data_dict

In [80]:
human_more_dropout = True

In [87]:
print(example['pts'].shape)
pts, gt_bboxes_3d, gt_labels = example['pts'], example['gt_bboxes_3d'], example['gt_labels']

gt_bbox3d_corner_pts = boxes_to_corners_3d(gt_bboxes_3d)

# randomly pick for each gt bbox one of the 8 octodrons to dropout
idx_octodron = np.random.randint(8, size=gt_bbox3d_corner_pts.shape[0])
corners_picked = gt_bbox3d_corner_pts[np.arange(gt_bbox3d_corner_pts.shape[0]), idx_octodron]

# construct the new centroids and new h, l, w dims of the filtering 3d bboxes
# the yaw around z axis remains unchanged
new_centers = (gt_bboxes_3d[:, :3] + corners_picked) / 2
new_dims = gt_bboxes_3d[:, 3:-1] / 2

filter_bboxes = np.hstack((new_centers, new_dims, gt_bboxes_3d[:, -1:]))

print(remove_points_in_boxes3d(pts, filter_bboxes).shape)

if human_more_dropout:
    idx_human = np.where((gt_labels == 0) | (gt_labels == 1))
    idx_additional_octodron = (idx_octodron[idx_human] + 1) % 8
    gt_bboxes_human = gt_bboxes_3d[idx_human]
    gt_bboxes_human_corner_pts = gt_bbox3d_corner_pts[idx_human]

    additional_corners_picked = gt_bboxes_human_corner_pts[np.arange(gt_bboxes_human_corner_pts.shape[0]), idx_additional_octodron]

    additional_new_centers = (gt_bboxes_human[:, :3] + additional_corners_picked) / 2
    additional_new_dims = gt_bboxes_human[:, 3:-1] / 2
    additional_filter_bboxes = np.hstack((additional_new_centers, additional_new_dims, gt_bboxes_human[:, -1:]))

    filter_bboxes = np.vstack((filter_bboxes, additional_filter_bboxes))
print(remove_points_in_boxes3d(pts, filter_bboxes).shape)


(18482, 4)
(17702, 4)
(17543, 4)


In [85]:
idx_human = np.where((example['gt_labels'] == 0) | (example['gt_labels'] == 1))
idx_additional_octodron = (idx_octodron[idx_human] + 1) % 8
gt_bboxes_human = gt_bboxes_3d[idx_human]
gt_bboxes_human_corner_pts = gt_bbox3d_corner_pts[idx_human]

additional_corners_picked = gt_bboxes_human_corner_pts[np.arange(gt_bboxes_human_corner_pts.shape[0]), idx_additional_octodron]

additional_new_centers = (gt_bboxes_human[:, :3] + additional_corners_picked) / 2
additional_new_dims = gt_bboxes_human[:, 3:-1] / 2
additional_filter_bboxes = np.hstack((additional_new_centers, additional_new_dims, gt_bboxes_human[:, -1:]))
additional_filter_bboxes

[[ 20.129112   -12.325342    -1.265914     0.7527133    0.8211418
    1.7595896    2.0418122 ]
 [ 32.24332    -24.223549    -1.5812825    0.51810133   0.9286723
    1.7400386   -2.20573   ]
 [  7.2893305   -3.5439627   -1.3222083    0.77226436   1.0068762
    1.7986916    2.9814105 ]
 [  9.772701   -13.867619    -1.504134     0.8797948    1.0850803
    1.7302631    3.039528  ]
 [ 12.386402    -5.621556    -1.2402389    0.4007954    0.32259145
    1.5934062    0.66952163]
 [ 19.107811    -1.1583707   -1.3960189    0.6354073    0.57675433
    1.749814     0.6806836 ]
 [  8.023365    -1.0456825   -1.5032831    0.5669789    0.7820398
    1.8280181    2.0823584 ]
 [ 15.175438    -1.3967205   -1.1907212    0.6354073    0.57675433
    1.749814     0.73859596]
 [ 13.705591    -9.122082    -1.6778046    0.57675433   1.8475691
    1.7791407   -1.1178255 ]
 [ 29.905968   -26.774754    -1.0820103    0.5963054    1.6618347
    1.6911612    2.240866  ]
 [  5.0557323  -19.456848    -1.8042651    0.48

array([[ 19.860786  , -12.250813  ,  -1.7058114 ,   0.37635666,
          0.4105709 ,   0.8797948 ,   2.0418122 ],
       [ 31.979576  , -24.190126  ,  -1.1462729 ,   0.25905067,
          0.46433616,   0.8700193 ,  -2.20573   ],
       [  7.1388845 ,  -3.2646723 ,  -0.87253535,   0.38613218,
          0.5034381 ,   0.8993458 ,   2.9814105 ],
       [ 10.019144  , -13.620171  ,  -1.9366999 ,   0.4398974 ,
          0.54254013,   0.86513156,   3.039528  ],
       [ 12.51502   ,  -5.622609  ,  -0.84188735,   0.2003977 ,
          0.16129573,   0.7967031 ,   0.66952163],
       [ 19.14052   ,  -0.9463463 ,  -1.8334725 ,   0.31770366,
          0.28837717,   0.874907  ,   0.6806836 ],
       [  8.263235  ,  -1.0735713 ,  -1.0462786 ,   0.28348944,
          0.3910199 ,   0.91400903,   2.0823584 ],
       [ 14.960905  ,  -1.3970525 ,  -1.6281745 ,   0.31770366,
          0.28837717,   0.874907  ,   0.73859596],
       [ 14.184005  ,  -9.049587  ,  -1.2330194 ,   0.28837717,
          0.9237

In [61]:
example['gt_names']

array(['Car', 'Truck', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car', 'Car',
       'Car', 'Car', 'Car', 'Pedestrian', 'Pedestrian', 'Pedestrian',
       'Pedestrian', 'Pedestrian', 'Pedestrian', 'Pedestrian',
       'Pedestrian', 'Cyclist', 'Cyclist', 'Cyclist', 'Cyclist',
       'Cyclist', 'Cyclist'], dtype='<U10')

In [68]:
example['gt_labels']

array([ 2, -1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  1,  1,  1,  1,  1])